In [ ]:
import numpy as np
import cv2 as c
from copy import deepcopy
from PIL import Image
import pytesseract as tez

def input_process(input_image):
    c.imshow("Input_image",input_image)
    Blurred = c.GaussianBlur(input_image, (5,5), 0)
    gray = c.cvtColor(Blurred, c.COLOR_BGR2GRAY)

    sobelx = c.Sobel(gray,c.CV_8U,1,0,ksize=3)
    ret2,thres_hold = c.threshold(sobelx,0,255,c.THRESH_BINARY+c.THRESH_OTSU)

    return thres_hold

def cleaning_plate(plate):
    print ("cleaning..")
    gray = c.cvtColor(plate, c.COLOR_BGR2GRAY)

    _, thresh = c.threshold(gray, 150, 255, c.THRESH_BINARY)
    im1,count,hierarchy = c.findContours(thresh.copy(),c.RETR_EXTERNAL, c.CHAIN_APPROX_NONE)

    if count:
        areas = [c.contourArea(c) for c in count]
        max_index = np.argmax(areas)

        max_cnt = count[max_index]
        max_cntArea = areas[max_index]
        x,y,w,h = c.boundingRect(max_cnt)

        if not ratioCheck(max_cntArea,w,h):
            return plate,None

        cleaned_final = thresh[y:y+h, x:x+w]
        #cv2.imshow("Function Test",cleaned_final)
        return cleaned_final,[x,y,w,h]

    else:
        return plate,None


def contour_extraction(thres_hold):
    element = c.getStructuringElement(shape=c.MORPH_RECT, ksize=(17, 3))
    morph_img_threshold = thres_hold.copy()
    c.morphologyEx(src=thres_hold, op=c.MORPH_CLOSE, kernel=element, dst=morph_img_threshold)
    c.imshow("inverted_image",morph_img_threshold)
    c.waitKey(0)

    im2,count, hierarchy= c.findContours(morph_img_threshold,mode=c.RETR_EXTERNAL,method=c.CHAIN_APPROX_NONE)
    return count


def validate_ratio(area, width, height):
    ratio = float(width) / float(height)
    if ratio < 1:
        ratio = 1 / ratio

    aspect = 4.7272
    min = 15*aspect*15  
    max = 125*aspect*125  
    rmin = 3
    rmax = 6

    if (area < min or area > max) or (ratio < rmin or ratio > rmax):
        return False
    return True

def check_White(plate):
    avg = np.mean(plate)
    if(avg>=115):
        return True
    else:
        return False

def Rotation_Ratio(rect):
    (x, y), (width, height), rect_angle = rect

    if(width>height):
        angle = -rect_angle
    else:
        angle = 90 + rect_angle

    if angle>15:
        return False

    if height == 0 or width == 0:
        return False

    area = height*width
    if not validate_ratio(area,width,height):
        return False
    else:
        return True



def cleaning_Reading(input_image,count):

    for i,j in enumerate(count):
        min_rect = c.minAreaRect(j)

        if Rotation_Ratio(min_rect):

            x,y,w,h = c.boundingRect(j)
            plate_img = img[y:y+h,x:x+w]


            if(check_White(plate_img)):
                #count+=1
                cleaning_plate, rect = cleaning_plate(plate_img)

                if rect:
                    x1,y1,w1,h1 = rect
                    x,y,w,h = x+x1,y+y1,w1,h1
                    c.imshow("Clean",cleaning_plate)
                    c.waitKey(0)
                    plate_im = Image.fromarray(cleaning_plate)
                    text = tez.image_to_string(plate_im, lang='eng')
                    print ("Text_Detected  : ",text)
                    img = c.rectangle(input_image,(x,y),(x+w,y+h),(0,255,0),2)
                    c.imshow("Plate_Detected ",input_image)
                    c.waitKey()


if __name__ == '__main__':
    print ("starting..")

    input_image = c.imread("C:/Users/welcome/Desktop/github setup/images/index.jpg")

    thres_hold = input_process(input_image)
    count = contour_extraction(thres_hold)


cleaning_Reading(input_image,count)


starting..
